# Un mapa interactivo de casos de COVID-19

(September 5th, 2021)

<p style='text-align: justify;'>
Tiempo después del inicio del estado de emergencia en Perú, la plataforma de <a href="https://www.datosabiertos.gob.pe/">datos abiertos</a> del país empezó a hacer pública la información asociada a los casos positivos de COVID-19, así como de las muertes atribuibles a la enfermedad. Aquí, voy a un ejercicio que generé un mapa de Leaflet en donde se muestren los contagiados por COVID-19 a nivel de distrito.
</p>

In [1]:
# standard
import time, json, webbrowser, os, warnings
# requirements
import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from fuzzywuzzy import fuzz
import folium
from folium import Map, Marker
from folium.plugins import MarkerCluster
from PIL import ImageFont
from jinja2 import Template
# settings
# --------------------------------------------------
warnings.filterwarnings('ignore')

## Paso 1. Selenium

<p style='text-align: justify;'>
La plataforma de datos abiertos brinda un API solo para algunas de las bases de datos. A la fecha, la información sobre los casos de COVID-19 no puede ser recuperada por esa vía; solo a través de un link que abre una ventana de diálogo para la descarga. Esta es una primera dificultad para la automatización, pero podemos abordarla mediante Selenium. Opcionalmente, podemos obviar este paso y descargar la información manualmente en la carpeta <code>./data/</code>.
</p>

In [ ]:
save_path = './data/'
link = 'https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa'
xpath = '//*[@id="data-and-resources"]/div/div/ul/li/div/span/a'

options = webdriver.ChromeOptions()
preferences = {
    'download.default_directory': save_path,
    'safebrowsing.enabled': True,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True
}
options.add_experimental_option('prefs', preferences)
service = Service(executable_path=ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)
driver.get(link)
driver.find_element(By.XPATH, xpath).click()

print('Waiting 20 seconds for downloading...')
for _ in tqdm.tqdm(range(2)):
    time.sleep(20)

driver.close()
print('Download complete')

<p style='text-align: justify;'>
Dado que estamos utilizando Selenium para extraer la información, este paso solo será posible siempre y cuando no se modifiquen <code>link</code> ni <code>xpath</code>. Asimismo, se encuentra el archivo <code>./data/positivos_covid.zip</code> con datos de prueba para el ejercicio (descomprimir).
</p>

## Paso 2. Pandas

### 2.1. CSV: Contagiados y problemas

<p style='text-align: justify;'>
Una vez almacenada la información localmente, vemos que este archivo es un CSV. Este archivo tiene dos detalles importantes que se convierten en problemas; los abordaremos separadamente.
</p>

In [7]:
file_path = './data/positivos_covid.csv'
dataset = pd.read_csv(file_path, sep=',', encoding='iso-8859-1')
dataset.head(3)

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
0,20210227,7320cabdc1aaca6c59014cae76a134e6,PASCO,PASCO,HUAYLLAY,PR,39.0,FEMENINO,20200923.0
1,20210227,e81602051997ace8340bb8c18fe24c65,JUNIN,CHUPACA,YANACANCHA,PR,48.0,FEMENINO,20200922.0
2,20210227,cecdbf10074dbc011ae05b3cbd320a6f,PASCO,OXAPAMPA,PUERTO BERMUDEZ,PR,49.0,FEMENINO,20200923.0


#### 2.1.1. Problema 1: Lima y Lima-Región*

<p style='text-align: justify;'>
Separaron el departamento de Lima en dos: Lima y Lima-Región. Para efectos del ejercicio, agruparemos ambos.
</p>

In [8]:
dataset.loc[dataset['DEPARTAMENTO'] == 'LIMA REGION', 'DEPARTAMENTO'] = 'LIMA'
print('Done.')

Done.


#### 2.1.2. Problema 2: Casos "EN INVESTIGACIÓN"

<p style='text-align: justify;'>
En algunos departamentos al parecer no se sabe exactamente a qué provincia o distrito pertenece la persona contagiada. Estas observaciones serán filtradas.
</p>

In [9]:
for c in ['PROVINCIA', 'DISTRITO']:
    condition = dataset[c] == 'EN INVESTIGACIÓN'
    dataset = dataset[~condition]
print('Done.')

Done.


### 2.2. Ubicación geográfica: distritos y coordenadas

<p style='text-align: justify;'>
La información sobre coordenadas geográficas de los distritos la recupero a través de un shapefile de <a href="https://www.geogpsperu.com/2014/03/base-de-datos-peru-shapefile-shp-minam.html">geogpsperu.com</a>. Esta información la almaceno en un archivo CSV y se encuentra en el repositorio.
</p>

In [10]:
coord_file = './data/COORDENADAS_DISTRITAL.csv'
coord = pd.read_csv(coord_file, sep=';', encoding='utf-8-sig')
coord.head(3)

,LATITUD,LONGITUD,IDDPTO,DEPARTAMENTO,IDPROV,PROVINCIA,IDDIST,DISTRITO,CAPITAL,CODCCPP,FUENTE
0,-9.634362,-75.466545,10,HUANUCO,1009,PUERTO INCA,100902,CODO DEL POZUZO,CODO DEL POZUZO,1,INEI
1,-9.001327,-74.866440,10,HUANUCO,1009,PUERTO INCA,100904,TOURNAVISTA,TOURNAVISTA,1,INEI
2,-8.824921,-75.047859,25,UCAYALI,2503,PADRE ABAD,250305,ALEXANDER VON HUMBOLDT,ALEXANDER VON HUMBOLDT,1,INEI


## Paso 3: Fuzzy Wuzzy

<p style='text-align: justify;'>
Si bien en la información sobre contagios que brinda datosabiertos.gob.pe se encuentran observaciones a nivel de distrito, no existe un identificador único de este (ubigeo). La única forma de realizar un matching entre la información de contagios y las coordenadas distritales es a través del nombre de los departamentos, provincias y distritos.
</p>

In [12]:
dataset.sort_values(by=['DEPARTAMENTO','PROVINCIA','DISTRITO'], inplace=True)
dataset['INDEX'] = list(zip(dataset['DEPARTAMENTO'], dataset['PROVINCIA'], dataset['DISTRITO']))

coord.sort_values(by=['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO'], inplace=True)
coord['INDEX'] = list(zip(coord['DEPARTAMENTO'], coord['PROVINCIA'], coord['DISTRITO']))

dataset_index = list(dataset['INDEX'].unique())
coord_index = list(coord['INDEX'].unique())

not_in_shp = []
_ = [not_in_shp.append(i) for i in coord_index if i not in dataset_index]

print('Cantidad de distritos con nombre diferente:', len(not_in_shp) - (len(coord_index) - len(dataset_index)))

Cantidad de distritos con nombre diferente: 105


<p style='text-align: justify;'>
Entonces, hay varios distritos que tienen el nombre escrito de forma diferente en alguna de las bases; pero en realidad sí se refieren al mismo. Esto significa que no existiría un matching perfecto si no hiciéramos ajustes en los nombres. Para superar esto, usaremos una combinación entre Fuzzy Wuzzy y la correción manual donde sea necesario.
</p>

### 3.1. String matching

<p style='text-align: justify;'>
En términos sencillos, Fuzzy Wuzzy permite obtener un score que indica qué tan similares son dos cadenas de caracteres. Con esta herramienta podemos encontrar nombres homogéneos entre las bases y luego hacer un matching adecuado, por ejemplo:
</p>

In [13]:
s = fuzz.ratio('LIMA PORTILLO MANANTAY', 'LIMA PORTILLO MANANTAI')
if s > 80:
    print("Score: {}. It's a match!".format(s))

Score: 95. It's a match!


#### 3.1.1. Corrección con umbral de puntaje

<p style='text-align: justify;'>
Inicialmente usaremos Fuzzy Wuzzy para automatizar la correción de nombres usando un umbral de 95 puntos.
</p>

In [14]:
for x, y, z in not_in_shp:
    for a, b, c in dataset_index:
        ratio = fuzz.ratio(str(y + ' ' + z), str(b + ' ' + c))
        if ratio >= 95:
            dataset.loc[dataset['PROVINCIA'] == b, 'PROVINCIA'] = y
            dataset.loc[dataset['DISTRITO'] == c, 'DISTRITO'] = z
        else:
            pass

#### 3.1.2. Correción manual de nombres

<p style='text-align: justify;'>
Algunos de los nombres no superan el umbral de 95 puntos, pero igualmente se refieren al mismo distrito. Estos necesariamente requieren ser ajustados manualmente. Corriendo el código anterior para un <code>ratio</code> mayor o igual a 80 se puede recuperar una lista de nombres, y entre ellos podemos identificar cuáles se deberíamos ajustar.
</p>

In [15]:
dataset.loc[dataset['PROVINCIA'] == 'NAZCA', 'PROVINCIA'] = 'NASCA'
dataset.loc[dataset['DISTRITO'] == 'HUAY HUAY', 'DISTRITO'] = 'HUAY-HUAY'
dataset.loc[dataset['DISTRITO'] == 'SAN FCO DE ASIS DE YARUSYACAN', 'DISTRITO'] = 'SAN FRANCISCO DE ASIS DE YARUSYACAN'
dataset.loc[dataset['DISTRITO'] == 'SONDOR', 'DISTRITO'] = 'SONDORILLO'
dataset.loc[dataset['DISTRITO'] == 'CORONEL GREGORIO ALBARRACIN L.', 'DISTRITO'] = 'CORONEL GREGORIO ALBARRACIN LANCHIPA'
dataset.loc[dataset['DISTRITO'] == 'NAZCA', 'DISTRITO'] = 'NASCA'
dataset.loc[dataset['DISTRITO'] == 'ANDRES AVELINO CACERES D.', 'DISTRITO'] = 'ANDRES AVELINO CACERES DORREGARAY'

dataset['INDEX'] = list(zip(dataset['DEPARTAMENTO'], dataset['PROVINCIA'], dataset['DISTRITO']))

dataset_index = list(dataset['INDEX'].unique())
coord_index = list(coord['INDEX'].unique())

not_in_shp = []
_ = [not_in_shp.append(c) for c in coord_index if c not in dataset_index]

print('Cantidad de distritos con nombre diferente:', len(not_in_shp) - (len(coord_index) - len(dataset_index)))

Cantidad de distritos con nombre diferente: 59


Aquí quedarían pendientes por ajustar más distritos; pero para fines del ejercicio estos serán ignorados.

## Paso 4: Información Georreferenciada

<p style='text-align: justify;'>
La información de contagios se encuentra a nivel de observación, por lo que la agruparemos a nivel distrital; esto redunda en simplemente contar cuántas observaciones caen en un distrito. Adicionalmente, obtendremos la edad promedio en cada uno de ellos. Luego haremos un join utilizando como index el nombre del departamento, provincia y distrito. Es por esto que homogeneizamos los nombres.
</p>

In [16]:
# Collapse información a nivel de distrito
collapse_data = dataset.groupby(['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO']).agg({'FECHA_CORTE': 'count', 'EDAD': 'mean'})
collapse_data.reset_index(inplace=True)

# collapseData nuevo index
collapse_data['INDEX'] = collapse_data[['DEPARTAMENTO','PROVINCIA', 'DISTRITO']].agg(' '.join, axis=1) 
collapse_data.set_index('INDEX', inplace=True)
collapse_data.drop(['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO'], axis=1, inplace=True)

# shapef nuevo index
coord['INDEX'] = coord[['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO']].agg(' '.join, axis=1)
coord.set_index('INDEX', inplace=True)

# Join de dataframes
df_georef = coord.join(collapse_data, how='left')

# Drop distritos sin datos
df_georef.dropna(inplace=True)

## Paso 5: Folium: Python, JavaScript, CSS & HTML

<p style='text-align: justify;'>
Aquí diseñaremos nuestro mapa interactivo. Aplicaremos el paquete Folium, algo de JavaScript, CSS y HTML para lograr el diseño.
</p>

### 5.1. Variables

In [17]:
latitude, longitude = df_georef['LATITUD'], df_georef['LONGITUD']
distrito, provincia, departamento = df_georef['DISTRITO'], df_georef['PROVINCIA'], df_georef['DEPARTAMENTO']
numero = df_georef['FECHA_CORTE']
edades = df_georef['EDAD']

### 5.2. JavaScript en acción

<p style='text-align: justify;'>
En nuestro mapa queremos mostrar el número de contagios por área, por lo que para modificar ese aspecto de forma dinámica debemos utilizar un poco de JavaScript y Python en conjunto.
</p>

In [18]:
class MarkerWithProps(Marker):
    
    _template = Template(
        '''
        {% macro script(this, kwargs) %}
        var {{this.get_name()}} = L.marker(
                [{{this.location[0]}}, {{this.location[1]}}],
                {
                    icon: new L.Icon.Default(),
                    {%- if this.draggable %}
                    draggable: true,
                    autoPan: true,
                    {%- endif %}
                    {%- if this.props %}
                    props : {{ this.props }}
                    {%- endif %}
                }
            ).addTo({{this._parent.get_name()}});
        {% endmacro %}
        '''
    )
    
    def __init__(
        self, location, popup=None, tooltip=None, icon=None, draggable=False, props=None
    ) -> None:
        super(MarkerWithProps, self).__init__(location=location,popup=popup,tooltip=tooltip,icon=icon,draggable=draggable)
        self.props = json.loads(json.dumps(props))

icon_create_function = (
    '''
    function(cluster) {
        var c = ' marker-cluster-';
        var markers = cluster.getAllChildMarkers();
        var sum = 0;
        
        for (var i = 0; i < markers.length; i++) {
            sum += markers[i].options.props.population;
        }
        
        var sum_total = sum;
        
        if (sum_total < 1600) {
            c += 'small';
        } else if (sum_total < 6000) {
            c += 'medium';
        } else {
            c += 'large';
        }
        
        return new L.DivIcon({ html: '<div><span style="font-size: 7pt">' + sum_total + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) });
    }
    '''
)

### 5.3. Mapa en Folium

<p style='text-align: justify;'>
Aquí generaré un mapa de Folium, usaré el diseño "Matrix" de <a href="https://www.jawg.io/en/">JawgMaps</a>. Es necesario un token de a <a href="https://www.jawg.io/lab/">JawgLab</a> para usarlo; sin embargo dejaré un diseño alternativo en caso no se tenga el token para que el código continúe sin errores.
</p>

In [19]:
# coordenadas de Perú
coord_pe = (-8.043040, -75.534517)

try:
    with open('./private/keys/JAWGMAP-API-KEY.txt', 'r') as f:
        token = f.read()
    used_tile = 'https://tile.jawg.io/jawg-matrix/{z}/{x}/{y}{r}.png?access-token='+'{token}'.format(token=token)
except:
    used_tile = 'https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png'

# mapa base
map_pe = Map(
    coord_pe, zoom_start = 5, tiles=used_tile,
    attr=(
        '''
        <a href="https://www.jawg.io/en/">JawgMaps</a>.
        Fuente: <a href="https://www.datosabiertos.gob.pe/">datosabiertos.gob.pe</a>.
        '''
    )
)

# marcadores en el mapa
contagio = MarkerCluster(icon_create_function=icon_create_function).add_to(map_pe)
font = ImageFont.truetype('times.ttf', 12)

def colorcode(c):
    if c in range(0, 1600):
        color, icon = 'blue', 'heartbeat'
    elif c in range(1601, 6000):
        color, icon = 'orange', 'exclamation-circle'
    else:
        color, icon = 'red', 'fa-ambulance'
    return color, icon

for lat, lon, dep, pro, dis, num, edad in zip(latitude,longitude, departamento, provincia, distrito, numero, edades):
    map_icon = folium.Icon(color=colorcode(num)[0], icon=colorcode(num)[1], prefix='fa')
    map_popup = folium.Popup(
        html='{}, {}, {}<br>Contagiados: {}<br>Edad promedio: {}'.format(dep,pro,dis, int(num), round(edad)),
        max_width=font.getsize(dep+pro+dis+' '*4)[0], min_width=font.getsize(dep + pro + dis + ' '*4)[0],
        sticky=True
    )
    map_tooltip = folium.Tooltip('{}, {}, {}<br>Contagiados: {}<br>Edad promedio: {}'.format(dep,pro,dis,int(num),round(edad)))
    
    contagio.add_child(MarkerWithProps(
        location=[lat, lon], icon=map_icon, popup=map_popup, tooltip=map_tooltip, props={'population': num}
    ))

### 5.4. CSS & HTML para leyendas

<p style='text-align: justify;'>
Uso el siguiente código para añadir leyendas personalizadas en nuestro mapa.
</p>

In [20]:
item_txt = '''<br> &nbsp; <i class="fa fa-map-marker fa-2x" style="color:{col}"></i> &nbsp; {item}'''
item_clu = '''<br> &nbsp; <i class="fa fa-circle-o fa-lg" aria-hidden="true" style="color:{col}"></i> &nbsp; {item}'''
itms_1 = item_txt.format(item='Menos de 1600', col='#82CAFA')
itms_2 = item_txt.format(item='Entre 1600 y 6000', col='orange')
itms_3 = item_txt.format(item='Más de 6000', col='red')
itms_4 = item_clu.format(item='Contagios en el área', col='green')

legend_html = (
    '''
    <div style="
    position: fixed;
    top: 125px; left: 20px; width: 150px; height: 100px; margin: 0 auto;
    border: 2px solid grey;
    z-index: 9999;
    background-color: white;
    opacity: 0.55;
    font-size: 10px;
    font-weight: bold;
    line-height: 12px;
    "> &nbsp;
    {itm_txt_4}
    {itm_txt_1}
    {itm_txt_2}
    {itm_txt_3}
    </div>
    '''.format(itm_txt_1=itms_1, itm_txt_2=itms_2, itm_txt_3=itms_3, itm_txt_4=itms_4)
)

title_html = (
    '''
    <div style="
    position: fixed;
    top: 80px; left: 10px; width: 260px;
    height: 35px; line-height: 35px; text-align: center;
    border: 2px solid grey;
    z-index: 9999;
    border-radius: 25px;
    background-color: white;
    opacity: 0.55;
    font-size: 12px;
    font-family: fantasy;
    "> 
    {title}
    </div>
    '''.format(title='COVID-19 en Perú: contagios por distrito')
)

map_pe.get_root().html.add_child(folium.Element(legend_html))
map_pe.get_root().html.add_child(folium.Element(title_html))

### 5.5. El mapa

<p style='text-align: justify;'>
Finalmente, tenemos nuestro mapa interactivo. Cada uno de los círculos muestra el número de contagiados en el área. Los markers representan un distrito y los colores van de acuerdo con la leyenda.
</p>

In [21]:
map_pe.save('./temp/COVID19.html')
map_pe

<p style='text-align: justify;'>
Para ver el mapa en una ventana nueva:
</p>

In [22]:
webbrowser.open(os.path.realpath('./temp/COVID19.html'), new=2)

True